In [1]:
import sys
sys.path.append("..")
from federated_learning.helper import utils
from federated_learning.FLNet import FLNet
import torch


In [17]:
import copy
model = FLNet().to('cpu')
m2 = FLNet().to('cpu')
mm = dict()
for ii in range(5):
    ii_w = "worker_{}".format(ii)
    mm[ii_w] = copy.deepcopy(model)
    print(id(mm[ii_w]))

print("")
print("Object: {}".format(id(mm)))    
print("---")

def test_(models, ss):
    print("Inside: {}".format(id(models)))
    for ii, mm in models.items():
        kk = mm
        kk.load_state_dict(utils.negative_parameters(ss))
        print(id(kk))

utils.print_model("m2", m2)
utils.print_model("Before", mm['worker_0'])
test_(mm, m2.state_dict())
utils.print_model("After", mm['worker_0'])

140280612179392
140277791200976
140277791306368
140277791307424
140277791308048

Object: 140277791149760
---

m2: tensor([-0.0544, -0.1179,  0.0903,  0.0507, -0.0198,  0.0619,  0.1493])


Before: tensor([-0.1228,  0.0365,  0.1013, -0.1470,  0.0612, -0.1635,  0.0814])

Inside: 140277791149760
140280612179392
140277791200976
140277791306368
140277791307424
140277791308048

After: tensor([ 0.0544,  0.1179, -0.0903, -0.0507,  0.0198, -0.0619, -0.1493])



In [8]:
utils.print_model('model', model)
# ss = model.state_dict()
aa = utils.negative_parameters(model.state_dict())


model: tensor([ 0.0645, -0.1190,  0.0336, -0.0810, -0.0881, -0.1217, -0.1938])



In [10]:
model_new = FLNet().to('cpu')
utils.print_model('New', model_new)


New: tensor([ 0.0166,  0.0261,  0.1727, -0.0657, -0.1804,  0.0308,  0.0728])



In [11]:
model_new.load_state_dict(aa)
utils.print_model('New', model_new)


New: tensor([-0.0645,  0.1190, -0.0336,  0.0810,  0.0881,  0.1217,  0.1938])



In [ ]:
class test():
    def __init__(self, param0=None, param1=None, param2=None):
        print("param2", param2)
        print("param1", param1)
        print("param0", param0)

In [ ]:
aa = test(args)

In [ ]:
def test1(parameter, **args):
    print(parameter)
    print(args.items())
    for ii, jj in args.items():
        print(ii, jj)

In [ ]:
def test2(parameter, *args):
    print(parameter)
    print(args)

In [ ]:
test1("ehsan", **args)

In [ ]:
test2("ehsan", ['ehsan','etesami', 1234])

In [ ]:
draw_samples(raw_train_data, eavesdroppers_idx[0])

In [ ]:
for ww in list(raw_train_data_new.keys()):
    raw_train_data_new[ww] = 0

In [ ]:
gg=dict()
gg['f0640_19']=dict()
gg['f0640_19']['y'] = raw_train_data['f0640_19']['y']
gg['f0640_19']['x'] = kk
draw_samples(gg)

In [ ]:
kk.shape

In [ ]:
raw_train_data.keys()

In [ ]:
# type(fl.workers['f0557_33'])
cc = torch.utils.data.DataLoader(bb, batch_size=len(bb))
# bb.federate([fl.workers['f0557_33']])

In [ ]:
img = torch.rand(784, dtype=float32)
img = img.reshape(1, 1, 28, 28)
img.shape
ff = FLNetComplex().to(torch.device('cpu'))
ff(img)

In [ ]:
fed_train_dataloader = fl.create_femnist_server_train_dataloader(
    train_dataset, configs['runtime']['server_w0_batch_size'])

# Create test dataloader from all normal and eveasdroppers
# fed_test_dataloader = fl.create_femnist_server_test_dataloader(raw_test_data, workers_idx_to_be_used)

In [ ]:
# Create test dataloader from all normal and eveasdroppers
fed_test_dataloader = fl.create_femnist_server_test_dataloader(raw_test_data, workers_idx_to_be_used)

In [ ]:
print(len(raw_train_data), len(raw_test_data))

In [ ]:
len(eavesdroppers_idx)

In [ ]:
len(workers_idx_all)

In [ ]:
for n, (a, b) in enumerate(test_dataloader):
    print("{}: {}, {}".format(n, a.shape, b.shape))

In [ ]:
utils.dataset_info(raw_train_data)
print()
utils.dataset_info(raw_test_data)

In [ ]:
flattened_data_x, flattened_data_y = utils.get_flattened_data(raw_test_data)

In [ ]:
import matplotlib.pyplot as plt
def draw_samples(data):
    list_keys = list(data.keys())
    data_np = dict()
    data_np['x'] = np.array(data[list_keys[0]]['x'], dtype = np.float32).reshape(-1, 28, 28)
    data_np['y'] = np.array(data[list_keys[0]]['y'], dtype = np.int64).reshape(-1, 1)
    figure = plt.figure(figsize=(8,8))
    for i in range(36):
        plt.subplot(6, 6, i + 1)
        plt.axis("off")
        plt.title(data_np['y'][i])
        plt.imshow(data_np['x'][i], cmap='gray')
        
draw_samples(raw_test_data)

In [ ]:
ww = ["worker_" + str(i) for i in range(20)]

In [ ]:
atk = utils.get_workers_idx(workers_idx, 5, [])

In [ ]:
atk

In [ ]:
bb = [ii for ii in ww if ii in atk]

In [ ]:
bb

In [ ]:
aa = [ii if ii in atk else 0 for ii in ww]

In [ ]:
d = [n for n in aa if n]

In [ ]:
d